In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd

In [11]:
# load csv
df = pd.read_csv("../outputs/2021-01-13/20-54-33/tensor.csv", header=None, delimiter="\t")
df_label = pd.read_csv("../outputs/2021-01-13/20-54-33/metadata.tsv", delimiter="\t", index_col=0)

In [12]:
data = pd.concat([df_label, df], axis=1)
del df, df_label

In [14]:
import scipy.spatial.distance

In [15]:
def cossim(x, y):
    return scipy.spatial.distance.cosine(x, y)

In [23]:
context_x = np.array(data.loc[data["ContextOrTargetOrShapeTarget"]==0].iloc[:, 2:])
target_x = np.array(data.loc[data["ContextOrTargetOrShapeTarget"]==1].iloc[:, 2:])
starget_x = np.array(data.loc[data["ContextOrTargetOrShapeTarget"]==2].iloc[:, 2:])

In [33]:
def mean_cossim(X=context_x, Y=target_x):
    tmp = 0.
    for i in range(context_x.shape[0]):
        x = X[i, :]
        y = Y[i, :]
        tmp += cossim(x, y)
    return tmp/context_x.shape[0]

In [41]:
print("context VS target \t\t {:.3f}".format(mean_cossim()))
print("context VS shape target \t {:.3f}".format(mean_cossim(context_x, starget_x)))
print("shape target VS target \t\t {:.3f}".format(mean_cossim(starget_x, target_x)))

context VS target 		 0.482
context VS shape target 	 0.001
shape target VS target 		 0.488
